# Random Forest (Data Set Too Big?)

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import missingno as msno
import os
import math
%matplotlib inline

In [3]:
from sklearn import ensemble
randomForest = ensemble.RandomForestClassifier(n_jobs=1)
dir(ensemble);

In [4]:
autos = pd.read_csv('autos_processed.csv', index_col = 0)
autos.head()

,name,seller,offerType,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,postalCode
1,A5_Sportback_2.7_Tdi,privat,Angebot,18300,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24,66954
2,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14,90480
3,GOLF_4_1_4__3T�RER,privat,Angebot,1500,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17,91074
4,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31,60437
5,BMW_316i___e36_Limousine___Bastlerfahrzeug__Ex...,privat,Angebot,650,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja,2016-04-04,33775


<br>

In [5]:
#function for visualizing model from lecture

def plot_model(model, x, y, label):
    '''
    model: a fitted model
    x, y: two variables, should arrays
    label: true label
    '''
    margin = 0.5
    x_min = x.min() - margin
    x_max = x.max() + margin
    y_min = y.min() - margin
    y_max = y.max() + margin
    import  matplotlib.pyplot as plt
    from matplotlib import colors
    col_dict = {
        'red': [(0, 1, 1), (1, 0.7, 0.7)],
        'green': [(0, 1, 0.5), (1, 0.7, 0.7)],
        'blue': [(0, 1, 0.5), (1, 1, 1)]
    }
    cmap = colors.LinearSegmentedColormap('red_blue_classes', col_dict)
    plt.cm.register_cmap(cmap=cmap)
    nx, ny = 200, 200
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, nx),
        np.linspace(y_min, y_max, ny)
    )
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ## plot colormap
    plt.pcolormesh(xx, yy, Z, cmap='red_blue_classes')
    ## plot boundaries
    plt.contour(xx, yy, Z, [0.5], linewidths=1., colors='k')
    plt.contour(xx, yy, Z, [1], linewidths=1., colors='k')
    ## plot scatters and true labels
    plt.scatter(x, y, c=label, edgecolors='k')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    ## if it's a SVM model
    if hasattr(model, 'support_'):
        # if it's a SVC, plot the support vectors
        index = model.support_
        plt.scatter(x[index], y[index], c=label[index], s=200, alpha=0.33, edgecolors='k')

<br> 

In [6]:
from sklearn import preprocessing #Need to convert strings with Label Encoder
le = preprocessing.LabelEncoder()

autostr_df = autos.drop(['price', 'kilometer','powerPS','yearOfRegistration','monthOfRegistration','postalCode'],axis=1)
col = autostr_df.columns

autos.loc[:,col] = autos.loc[:,col].apply(le.fit_transform)


D:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [7]:
autos.head(1)

,name,seller,offerType,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,postalCode
1,2712,0,0,18300,4,2011,2,190,0,125000,5,4,1,1,95,66954


In [8]:
autos.dateCreated = pd.to_datetime(autos.dateCreated)
x = autos.drop(['price'],axis=1)
x.dateCreated = pd.to_numeric(x.dateCreated)
y = autos.price

In [9]:
import sklearn.model_selection as ms

In [10]:
#Train Test Split takes too much Memory even with a 40-60 split

x_train, x_test, y_train, y_test = ms.train_test_split(x,y, 
                                                       test_size=0.3, random_state=0)
print('Original: {}, {}'.format(x.shape, y.shape))
print('Training: {}, {}'.format(x_train.shape, y_train.shape))
print('Test: {}, {}'.format(x_test.shape, y_test.shape))

Original: (290916, 15), (290916L,)
Training: (203641, 15), (203641L,)
Test: (87275, 15), (87275L,)


In [11]:
# #K-fold

# import sklearn.model_selection as ms

# print('All data: {}'.format(autos))
# ms_k3s = ms.StratifiedKFold(n_splits=10)
# # for train_idx, val_idx in ms_k3.split(autos):
# #     print('Train:', train_idx, 'Validation:', val_idx)

In [12]:
# import sklearn.cross_validation as cv
# cv_k3s = cv.StratifiedKFold(y, n_folds=20)
# # for train_idx, val_idx in cv_k3s:
# #     print('Train:', y[train_idx], 'Validation: ', y[val_idx])

In [13]:
# randomForest.fit(x, y) # fit
# randomForest.score(x, y) # accuracy


In [24]:
plot_model(randomForest, autos.kilometer, autos.powerPS, autos.price)
plt.xlabel('km')
plt.ylabel('powerPS')
plt.show()

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.

<br>
## XGBoost
<br>

In [ ]:
import xgboost as xgb

def runXGB(x_train, y_train, num_rounds=900):
        params = {}
        #params here
    
        plst = list(params.items())  
        num_rounds = num_rounds

        xgtrain = xgb.DMatrix(np.array(x_train), label=np.array(y_train))
        cvresult = xgb.cv(params, xgtrain, num_boost_round=num_rounds, nfold=5)
        
        
        print("number of rounds %d" %(cvresult.ix[:,'test-rmse-mean'].argmin()+1))
        num_rounds = cvresult.ix[:,'test-rmse-mean'].argmin()+1


        model = xgb.train(plst, xgtrain, num_rounds)
        return(model)

def predictXGB(model, X):
    X_dm = xgb.DMatrix(np.array(X))
    return(model.predict(X_dm))

def rmse(Y1, Y2):
    
    return(np.sqrt(np.mean((Y1-Y2)**2)))

def mape(Y1, Y2):
    return(np.mean(np.abs(Y1-Y2)/(np.abs(Y2))))